In [3]:
#https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b
import importlib
import pandas as pd
import nltk
from glove import Corpus, Glove
import logging

import utils
importlib.reload(utils)

<module 'utils' from '/home/andryw/Projects/tutorials/embeddings/utils.py'>

In [10]:
unlabel_data = pd.read_csv("data/unlabeledTrainData.tsv", delimiter="\t", quoting=3)
train_data = pd.read_csv("data/labeledTrainData.tsv", delimiter="\t", quoting=3)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

punkt_tokenizer = nltk.load("tokenizers/punkt/english.pickle")

In [20]:
reviews = list(unlabel_data["review"]) + list(train_data["review"])

sentences = []
for r in reviews:
    sentences += utils.text_to_sentences(r, punkt_tokenizer)
    
sentences_word_list = [utils.normalize_text(sentence).split() for sentence in sentences]

In [18]:
corpus = Corpus()

In [22]:
corpus.fit(sentences_word_list, window=10)

In [23]:
glove = Glove(no_components=100)

In [36]:
glove.fit(corpus.matrix, epochs=5, no_threads=3, verbose=True)

Performing 5 training epochs with 3 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


### Load data to predict

In [46]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import xgboost as xgb

In [47]:
data = pd.read_csv("data/labeledTrainData.tsv", delimiter="\t", quoting=3)
train_data, test_data = train_test_split(data, test_size=0.25)

In [48]:
train_data['review_cleaned'] = train_data.review.apply(lambda x: utils.normalize_text(x))
test_data['review_cleaned'] = test_data.review.apply(lambda x: utils.normalize_text(x))

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
def get_avg_vectors(text, model):
    total_words = 0
    final_vector = np.zeros(int(glove.no_components))
    for word in text.split(" "):
        if word in model.dictionary:
            final_vector += model.word_vectors[model.dictionary[word]]
            total_words+=1
    return final_vector / total_words

train_vectors = [get_avg_vectors(x, glove) for x in train_data['review_cleaned']]
train_vectors=np.array(train_vectors)

test_vectors = [get_avg_vectors(x, glove) for x in test_data['review_cleaned']]
test_vectors=np.array(test_vectors)

In [55]:
x_train = train_vectors
y_train = train_data.sentiment

x_test = test_vectors
y_test  = test_data.sentiment

In [56]:
logistic_regression = LogisticRegression(verbose=True, n_jobs=3)
logistic_regression = logistic_regression.fit(x_train, y_train)
metrics.roc_auc_score(logistic_regression.predict(x_test), y_test)

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]

0.8025269653249758

In [57]:
# params = {"n_jobs": 3, "silent": True, "n_estimators": 100, 'objective':'binary:logistic'}
# xgboost_classifier = xgb.XGBClassifier(**params)

# xgboost_classifier.fit(x_train, y_train, verbose=True, eval_metric='auc',
#                       eval_set=[(x_train, y_train), (x_test, y_test)],)

# metrics.roc_auc_score(xgboost_classifier.predict(x_test), y_test)

## Predict

In [60]:
unlabel_data = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabel_data['review_cleaned'] = unlabel_data.review.apply(lambda x: utils.normalize_text(x))
unlabel_vectors = [get_avg_vectors(x, glove) for x in unlabel_data['review_cleaned']]
unlabel_vectors=np.array(unlabel_vectors)

In [61]:
utils.save_predict(unlabel_data, 
                   logistic_regression.predict(unlabel_vectors), 
                   "logistic_glove_window-10_without-stopwords.csv")